In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master('local').appName('zero_analyst').getOrCreate()
spark

In [2]:
# MySQL connection properties
mysql_host = "localhost"
mysql_port = 3306
mysql_database = "zero_analyst"
mysql_username = "root"
mysql_password = "mypassword"

# JDBC URL
jdbc_url = f"jdbc:mysql://{mysql_host}:{mysql_port}/{mysql_database}"

In [3]:
# 01 Write the SQL query to find the second highest salary   SELECT * FROM employees01 
employees01 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'employees01').option("user", mysql_username).option("password", mysql_password).load()

employees01.show()
# -- m1
from pyspark.sql import Window
from pyspark.sql.functions import col,dense_rank
ws=Window.partitionBy('department').orderBy(col('salary').desc())

employees01.withColumn('dnk',dense_rank().over(ws)).filter(col('dnk')<2).show()

+-----------+---------------+-----------+--------+
|employee_id|           name| department|  salary|
+-----------+---------------+-----------+--------+
|          1|       John Doe|Engineering|63000.00|
|          2|     Jane Smith|Engineering|55000.00|
|          3|Michael Johnson|Engineering|64000.00|
|          4|    Emily Davis|  Marketing|58000.00|
|          5|    Chris Brown|  Marketing|56000.00|
|          6|    Emma Wilson|  Marketing|59000.00|
|          7|       Alex Lee|      Sales|58000.00|
|          8|    Sarah Adams|      Sales|58000.00|
|          9|     Ryan Clark|      Sales|61000.00|
|         11|           zara|         it|63000.00|
+-----------+---------------+-----------+--------+

+-----------+---------------+-----------+--------+---+
|employee_id|           name| department|  salary|dnk|
+-----------+---------------+-----------+--------+---+
|          3|Michael Johnson|Engineering|64000.00|  1|
|          6|    Emma Wilson|  Marketing|59000.00|  1|
|         

In [18]:
# 02 write an SQL query to calculate the total  numbers of returned orders for each month select * from orders02; select * from returns02;
orders02 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'orders02').option("user", mysql_username).option("password", mysql_password).load()
returns02 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'returns02').option("user", mysql_username).option("password", mysql_password).load()
from pyspark.sql.functions import count,month
orders02.show()
returns02.show()

orders02.join(returns02,'OrderID','left').groupby(month('OrderDate')).agg(count(col('ReturnID'))).show()


+-------+----------+-----------+
|OrderID| OrderDate|TotalAmount|
+-------+----------+-----------+
|      1|2023-01-15|     150.50|
|      2|2023-02-20|     200.75|
|      3|2023-02-28|     300.25|
|      4|2023-03-10|     180.00|
|      5|2023-04-05|     250.80|
+-------+----------+-----------+

+--------+-------+
|ReturnID|OrderID|
+--------+-------+
|     104|      1|
|     101|      2|
|     105|      3|
|     102|      4|
|     103|      5|
+--------+-------+

+----------------+---------------+
|month(OrderDate)|count(ReturnID)|
+----------------+---------------+
|               1|              1|
|               3|              1|
|               4|              1|
|               2|              2|
+----------------+---------------+



In [21]:
# 03 Write SQL query to find the top-selling products in each category   SELECT * FROm products03;
products03 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'products03').option("user", mysql_username).option("password", mysql_password).load()

products03.show()
ws=Window.partitionBy('category').orderBy(col('quantity_sold').desc())
products03.withColumn('dnk',dense_rank().over(ws)).filter(col('dnk')==1).show()

+----------+--------------------+--------------+-------------+
|product_id|        product_name|      category|quantity_sold|
+----------+--------------------+--------------+-------------+
|         1|  Samsung Galaxy S20|   Electronics|          100|
|         2| Apple iPhone 12 Pro|   Electronics|          150|
|         3|  Sony PlayStation 5|   Electronics|           80|
|         4|    Nike Air Max 270|      Clothing|          200|
|         5|Adidas Ultraboost 20|      Clothing|          200|
|         6|Levis Mens 501 Jeans|      Clothing|           90|
|         7|Instant Pot Duo 7...|Home & Kitchen|          180|
|         8|Keurig K-Classic ...|Home & Kitchen|          130|
|         9|iRobot Roomba 675...|Home & Kitchen|          130|
|        10|Breville Compact ...|Home & Kitchen|           90|
|        11|Dyson V11 Animal ...|Home & Kitchen|           90|
+----------+--------------------+--------------+-------------+

+----------+--------------------+--------------+------

In [4]:
# 04 Find the top 2 products in the top 2 categories based on spend amount? SELECT * FROM orders04;
orders04 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'orders04').option("user", mysql_username).option("password", mysql_password).load()
orders04.show()
from pyspark.sql.functions import *
#orders04.printSchema()
#ws=Window.partitionBy('category').orderBy(col('spend').desc())
cat_df=orders04.groupby('category').agg(sum('spend')).orderBy(col('sum(spend)').desc()).select('category').limit(2)
cat_df.show()

cat_df.join(orders04,'category','inner').groupBy('product','category').agg(sum(col('spend'))).orderBy(col('sum(spend)').desc()).show(4)

+-----------+---------------+-------+-----+----------------+
|   category|        product|user_id|spend|transaction_date|
+-----------+---------------+-------+-----+----------------+
|  appliance|   refrigerator|    165|  246|      2021-12-26|
|  appliance|   refrigerator|    123|  300|      2022-03-02|
|  appliance| washingmachine|    123|  220|      2022-03-02|
|electronics|         vacuum|    178|  152|      2022-04-05|
|electronics|wirelessheadset|    156|  250|      2022-07-08|
|electronics|             TV|    145|  189|      2022-07-15|
| Television|             TV|    165|  129|      2022-07-15|
| Television|             TV|    163|  129|      2022-07-15|
| Television|             TV|    141|  129|      2022-07-15|
|       toys|          Ben10|    145|  189|      2022-07-15|
|       toys|          Ben10|    145|  189|      2022-07-15|
|       toys|           yoyo|    165|  129|      2022-07-15|
|       toys|           yoyo|    163|  129|      2022-07-15|
|       toys|           

In [68]:
# 06 -- who has not done purchase in last month (orders)  SELECT * FROM customers06;SELECT * FROM orders06;
customers06 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'customers06').option("user", mysql_username).option("password", mysql_password).load()
orders06 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'orders06').option("user", mysql_username).option("password", mysql_password).load()
customers06.show()
orders06.show()

from pyspark.sql.functions import col, year, month, current_date, expr
df_current = spark.createDataFrame([(1,)], ["dummy"]) \
                  .withColumn("current_year", year(current_date())) \
                  .withColumn("last_month", expr("IF(month(current_date())=1, 12, month(current_date())-1)"))

current_year = df_current.collect()[0]["current_year"]
last_month = df_current.collect()[0]["last_month"]

# Filter orders from the last month
last_month_orders = orders06.filter(
    (year(col("order_date")) == current_year) & 
    (month(col("order_date")) == last_month)
).select("customer_id").distinct()

# Find customers who have NOT made a purchase in the last month using LEFT ANTI JOIN
inactive_customers = customers06.join(last_month_orders, "customer_id", "left_anti")

# Show results
inactive_customers.show()

+-----------+-------------+-----------------+
|customer_id|         name|            email|
+-----------+-------------+-----------------+
|          1|     John Doe| john@example.com|
|          2|   Jane Smith| jane@example.com|
|          3|Alice Johnson|alice@example.com|
|          4|        Sam B|   sb@example.com|
|          5|   John Smith|    j@example.com|
+-----------+-------------+-----------------+

+--------+-----------+----------+------+
|order_id|customer_id|order_date|amount|
+--------+-----------+----------+------+
|       1|          1|2024-03-05| 50.00|
|       2|          2|2024-03-10| 75.00|
|       5|          4|2024-04-02| 45.00|
|       5|          2|2024-04-02| 45.00|
|       3|          4|2024-04-15|100.00|
|       4|          1|2024-04-01| 60.00|
|       5|          5|2024-04-02| 45.00|
|       5|          5|2024-09-02| 45.00|
|       6|          1|2024-04-05|100.00|
|       6|          1|2025-01-10| 10.00|
|       7|          2|2025-01-07| 10.00|
+--------+-

In [67]:
# 07   How would you identify duplicate entries in a SQL in given table employees columns are  emp_id, name, department, salary:
employees07 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'employees07').option("user", mysql_username).option("password", mysql_password).load()
employees07.show()

employees07.groupby('name').agg(count('name')).filter(col('count(name)')>1).show()

+------+--------------+----------+--------+
|emp_id|          name|department|  salary|
+------+--------------+----------+--------+
|     1|      John Doe|   Finance|60000.00|
|     2|    Jane Smith|   Finance|65000.00|
|     2|    Jane Smith|   Finance|65000.00|
|     9| Lisa Anderson|     Sales|63000.00|
|     9| Lisa Anderson|     Sales|63000.00|
|     9| Lisa Anderson|     Sales|63000.00|
|    10|Kevin Martinez|     Sales|61000.00|
+------+--------------+----------+--------+

+-------------+-----------+
|         name|count(name)|
+-------------+-----------+
|Lisa Anderson|          3|
|   Jane Smith|          2|
+-------------+-----------+



In [70]:
# 08   Write a  query to find all products that have not been sold in the last six months. 
# Return the product_id, product_name, category,  and price of these products. SELECT * FROM products08;SELECT * FROM sales08;
products08 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'products08').option("user", mysql_username).option("password", mysql_password).load()
sales08 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'sales08').option("user", mysql_username).option("password", mysql_password).load()
products08.show()
sales08.show()
from pyspark.sql.functions import col, current_date, date_sub
six_months_ago = date_sub(current_date(), 6 * 30)  # Approximate 6 months as 180 days

# Left join products with sales to find unsold products or those last sold more than 6 months ago
unsold_products = products08.join(sales08, "product_id", "left") \
    .filter((col("sale_date").isNull()) | (col("sale_date") < six_months_ago)) \
    .select(products08["*"])  # Selecting required columns

# Show results
unsold_products.show()

+----------+------------+----------+-----+
|product_id|product_name|  category|price|
+----------+------------+----------+-----+
|         1|   Product A|Category 1|10.00|
|         2|   Product B|Category 2|15.00|
|         3|   Product C|Category 1|20.00|
|         4|   Product D|Category 3|25.00|
+----------+------------+----------+-----+

+-------+----------+----------+--------+
|sale_id|product_id| sale_date|quantity|
+-------+----------+----------+--------+
|      1|         1|2023-09-15|       5|
|      2|         2|2023-10-20|       3|
|      3|         1|2024-01-05|       2|
|      4|         3|2024-02-10|       4|
|      5|         4|2023-12-03|       1|
|      6|         1|2024-12-30|      10|
|      7|         3|2024-11-30|     100|
|      8|         2|2024-10-30|       5|
|      9|         3|2024-09-30|       5|
+-------+----------+----------+--------+

+----------+------------+----------+-----+
|product_id|product_name|  category|price|
+----------+------------+----------

In [54]:
# 09  write a SQL query to find customers who  bought Airpods after purchasing an iPhone. SELECT * FROM customers09; SELECT * FROM purchases09;
customers09 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'customers09').option("user", mysql_username).option("password", mysql_password).load()
purchases09 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'purchases09').option("user", mysql_username).option("password", mysql_password).load()
customers09.show()
purchases09.show()

p1 = purchases09.alias("p1").withColumnRenamed('ProductName','p1_ProductName').withColumnRenamed('PurchaseDate','p1_PurchaseDate').filter(col("ProductName") == "iPhone")
p2 = purchases09.alias("p2").withColumnRenamed('ProductName','p2_ProductName').withColumnRenamed('PurchaseDate','p2_PurchaseDate').filter(col("ProductName") == "Airpods")
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")
#p1.show()
purchases_joined = customers09.join(p1,'CustomerID','inner').join(p2,'CustomerID','inner').filter(p1.p1_PurchaseDate < p2.p2_PurchaseDate).select('CustomerID','CustomerName')
purchases_joined.show()

+----------+------------+
|CustomerID|CustomerName|
+----------+------------+
|         1|        John|
|         2|        Emma|
|         3|     Michael|
|         4|         Ben|
|         5|        John|
+----------+------------+

+----------+----------+-----------+------------+
|PurchaseID|CustomerID|ProductName|PurchaseDate|
+----------+----------+-----------+------------+
|       100|         1|     iPhone|  2024-01-01|
|       101|         1|    MacBook|  2024-01-20|
|       102|         1|    Airpods|  2024-03-10|
|       103|         2|       iPad|  2024-03-05|
|       104|         2|     iPhone|  2024-03-15|
|       105|         3|    MacBook|  2024-03-20|
|       106|         3|    Airpods|  2024-03-25|
|       107|         4|     iPhone|  2024-03-22|
|       108|         4|    Airpods|  2024-03-29|
|       110|         5|    Airpods|  2024-02-29|
|       109|         5|     iPhone|  2024-03-22|
+----------+----------+-----------+------------+

+----------+------------+
|Cu

In [11]:
# 10 Write a SQL query to classify employees into three categories based on their salary:employees10
employees10 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'employees10').option("user", mysql_username).option("password", mysql_password).load()
from pyspark.sql.functions import when,count,lit
employees10.show()
employees10.withColumn("grading",
                       when(col('Salary') >=  75000,"A1")
                       .when((col('Salary')>= 50000) & (col('Salary') < 75000),"A2")
                       .otherwise("A3")
                       ).show()
                        
                        

+----------+---------+--------+----------+--------+
|EmployeeID|FirstName|LastName|Department|  Salary|
+----------+---------+--------+----------+--------+
|         1|     John|     Doe|   Finance|75000.00|
|         2|     Jane|   Smith|        HR|60000.00|
|         3|  Michael| Johnson|        IT|45000.00|
|         4|    Emily|   Brown| Marketing|55000.00|
|         5|    David|Williams|   Finance|80000.00|
|         6|    Sarah|   Jones|        HR|48000.00|
|         7|    Chris|  Taylor|        IT|72000.00|
|         8|  Jessica|  Wilson| Marketing|49000.00|
+----------+---------+--------+----------+--------+

+----------+---------+--------+----------+--------+-------+
|EmployeeID|FirstName|LastName|Department|  Salary|grading|
+----------+---------+--------+----------+--------+-------+
|         1|     John|     Doe|   Finance|75000.00|     A1|
|         2|     Jane|   Smith|        HR|60000.00|     A2|
|         3|  Michael| Johnson|        IT|45000.00|     A3|
|         4|   

In [14]:
# 11 Identify returning customers based on their order history. Categorize customers as Returning if they have placed more than one return,  and as New otherwise. 
orders11 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'orders11').option("user", mysql_username).option("password", mysql_password).load()
returns11 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'returns11').option("user", mysql_username).option("password", mysql_password).load()
orders11.show()
returns11.show()

customers_count=orders11.join(returns11,'order_id','inner').groupby('customer_id').agg(count("*").alias("orders_count"))
customers_count.withColumn("customer_type",when(col("orders_count")>1,lit("returning")).otherwise(lit("new")) ).show()

+--------+-----------+----------+----------+--------+
|order_id|customer_id|order_date|product_id|quantity|
+--------+-----------+----------+----------+--------+
|    1001|       C001|2023-01-15|      P001|       4|
|    1002|       C001|2023-02-20|      P002|       3|
|    1003|       C002|2023-03-10|      P003|       8|
|    1004|       C003|2023-04-05|      P004|       2|
|    1005|       C004|2023-05-20|      P005|       3|
|    1006|       C002|2023-06-15|      P001|       6|
|    1007|       C003|2023-07-20|      P002|       1|
|    1008|       C004|2023-08-10|      P003|       2|
|    1009|       C005|2023-09-05|      P002|       3|
|    1010|       C001|2023-10-20|      P002|       1|
+--------+-----------+----------+----------+--------+

+---------+--------+
|return_id|order_id|
+---------+--------+
|     R001|    1001|
|     R002|    1002|
|     R003|    1005|
|     R004|    1008|
|     R005|    1007|
+---------+--------+

+-----------+------------+-------------+
|customer_id

In [16]:
# 13 write a SQL query to retrieve all employees  details along with their managers names based on the manager ID
employees13 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'employees13').option("user", mysql_username).option("password", mysql_password).load()
employees13.show()

joined_df = employees13.alias("e").join(
    employees13.alias("m"), 
    col("e.manager_id") == col("m.emp_id"), 
    "left"
).select(
    col("e.*"), 
    col("m.name").alias("manager_name")  # Renaming manager's name
)

# Show result
joined_df.show()

+------+--------------+----------+
|emp_id|          name|manager_id|
+------+--------------+----------+
|     1|      John Doe|      NULL|
|     2|    Jane Smith|         1|
|     3| Alice Johnson|         1|
|     4|     Bob Brown|         3|
|     5|   Emily White|      NULL|
|     6|   Michael Lee|         3|
|     7|   David Clark|      NULL|
|     8|   Sarah Davis|         2|
|     9|  Kevin Wilson|         2|
|    10|Laura Martinez|         4|
+------+--------------+----------+

+------+--------------+----------+-------------+
|emp_id|          name|manager_id| manager_name|
+------+--------------+----------+-------------+
|     1|      John Doe|      NULL|         NULL|
|     5|   Emily White|      NULL|         NULL|
|     7|   David Clark|      NULL|         NULL|
|     2|    Jane Smith|         1|     John Doe|
|     3| Alice Johnson|         1|     John Doe|
|     4|     Bob Brown|         3|Alice Johnson|
|     6|   Michael Lee|         3|Alice Johnson|
|    10|Laura Marti

In [20]:
# 14 Find the top 2 customers who have spent the most money across all their orders.
orders14 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'orders14').option("user", mysql_username).option("password", mysql_password).load()
customers14 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'customers14').option("user", mysql_username).option("password", mysql_password).load()
orders14.show()
customers14.show()
from pyspark.sql.functions import sum

join_df=customers14.join(orders14,'customer_id','inner')
join_df.show()
join_df.groupBy('customer_id','customer_name').agg(sum(col('order_amount'))).show()

+--------+-----------+----------+------------+
|order_id|customer_id|order_date|order_amount|
+--------+-----------+----------+------------+
|       1|          1|2024-01-03|       50.00|
|       2|          2|2024-01-05|       75.00|
|       3|          1|2024-01-10|       25.00|
|       4|          3|2024-01-15|       60.00|
|       5|          2|2024-01-20|       50.00|
|       6|          1|2024-02-01|      100.00|
|       7|          2|2024-02-05|       25.00|
|       8|          3|2024-02-10|       90.00|
|       9|          1|2024-02-15|       50.00|
|      10|          2|2024-02-20|       75.00|
+--------+-----------+----------+------------+

+-----------+-------------+-----------------+
|customer_id|customer_name|   customer_email|
+-----------+-------------+-----------------+
|          1|     John Doe| john@example.com|
|          2|   Jane Smith| jane@example.com|
|          3|Alice Johnson|alice@example.com|
|          4|    Bob Brown|  bob@example.com|
+-----------+------

In [33]:
# 15 Write an SQL query to retrieve the product details for items whose revenue decreased compared to the previous month. 
orders15 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'orders15').option("user", mysql_username).option("password", mysql_password).load()
orders15.show()
from pyspark.sql.functions import year,current_date,month

prev_month_revenue=orders15.filter((year('order_date')==year(current_date()))&(month('order_date')==month(current_date())-1)).groupBy(col('product_id')).agg(sum(col('quantity')).alias('prev_net_quantity'),sum(col('quantity')*col('price')).alias('prev_net_revenue'))
prev_month_revenue.show()

current_month_revenue=orders15.filter((year('order_date')==year(current_date()))&(month('order_date')==month(current_date()))).groupBy(col('product_id')).agg(sum(col('quantity')).alias('current_net_quantity'),sum(col('quantity')*col('price')).alias('current_net_revenue'))
current_month_revenue.show()

current_month_revenue.join(prev_month_revenue,'product_id','inner').show()

+--------+----------+----------+--------+-----+
|order_id|order_date|product_id|quantity|price|
+--------+----------+----------+--------+-----+
|       1|2024-04-01|         1|      10|50.00|
|       2|2024-04-02|         2|       8|40.00|
|       3|2024-04-03|         3|      15|30.00|
|       4|2024-04-04|         4|      12|25.00|
|       5|2024-04-05|         5|       5|60.00|
|       6|2024-04-06|         6|      20|20.00|
|       7|2024-04-07|         7|      18|35.00|
|       8|2024-04-08|         8|      14|45.00|
|       9|2024-04-09|         1|      10|50.00|
|      10|2024-04-10|         2|       8|40.00|
|      11|2024-03-01|         1|      12|50.00|
|      12|2024-03-02|         2|      10|40.00|
|      13|2024-03-03|         3|      18|30.00|
|      14|2024-03-04|         4|      14|25.00|
|      15|2024-03-05|         5|       7|60.00|
|      16|2024-03-06|         6|      22|20.00|
|      17|2024-03-07|         7|      20|35.00|
|      18|2024-03-08|         8|      16

In [36]:
# 16 Write a SQL query to find the names of managers who have at least five direct reports.Return the result table in any order. 

employees16 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'employees16').option("user", mysql_username).option("password", mysql_password).load()
employees16.show()

employees16.groupBy('managerId').agg(count('id').alias('no_of_employees')).show()

+---+-------+----------+---------+
| id|   name|department|managerId|
+---+-------+----------+---------+
|101|   John|         A|     NULL|
|102|    Dan|         A|      101|
|103|  James|         A|      101|
|104|    Amy|         A|      101|
|105|   Anne|         A|      101|
|106|    Ron|         B|      101|
|107|Michael|         C|     NULL|
|108|  Sarah|         C|      107|
|109|  Emily|         C|      107|
|110|  Brian|         C|      107|
+---+-------+----------+---------+

+---------+---------------+
|managerId|no_of_employees|
+---------+---------------+
|      101|              5|
|     NULL|              2|
|      107|              3|
+---------+---------------+



In [ ]:
# 17 Write an SQL query to find customers who have made purchases in all product categories. 
customers17 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'customers17').option("user", mysql_username).option("password", mysql_password).load()

purchases17 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'purchases17').option("user", mysql_username).option("password", mysql_password).load()
customers17.show()

purchases17.show()

from pyspark.sql.functions import countDistinct
total_products=purchases17.select(countDistinct('product_category')).collect()[0][0]
print(total_products)
type(total_products)
grouped_df=purchases17.groupBy('customer_id').agg(countDistinct('product_category').alias('distinct_category_count'))
filtered_df=grouped_df.filter(col('distinct_category_count')==total_products)
customers17.join(filtered_df,'customer_id','inner').show()

+-----------+-------------+
|customer_id|customer_name|
+-----------+-------------+
|          1|        Alice|
|          2|          Bob|
|          3|      Charlie|
|          4|        David|
|          5|         Emma|
+-----------+-------------+

+-----------+-----------+----------------+
|purchase_id|customer_id|product_category|
+-----------+-----------+----------------+
|        101|          1|     Electronics|
|        102|          1|           Books|
|        103|          1|        Clothing|
|        104|          1|     Electronics|
|        105|          2|        Clothing|
|        106|          1|          Beauty|
|        107|          3|     Electronics|
|        108|          3|           Books|
|        109|          4|           Books|
|        110|          4|        Clothing|
|        111|          4|          Beauty|
|        112|          5|     Electronics|
|        113|          5|           Books|
+-----------+-----------+----------------+

4
+-----------+